<a href="https://colab.research.google.com/github/SM-1001/INNA-for-DeepLearning/blob/master/Nadian_Keras_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Can't run easily. must be set the environment a few times!

In [1]:
!git clone https://github.com/SM-1001/INNA-for-DeepLearning.git

Cloning into 'INNA-for-DeepLearning'...
remote: Enumerating objects: 689, done.
remote: Counting objects: 100% (583/583), done.
remote: Compressing objects: 100% (233/233), done.
remote: Total 689 (delta 377), reused 542 (delta 349), pack-reused 106
Receiving objects: 100% (689/689), 138.16 KiB | 8.13 MiB/s, done.
Resolving deltas: 100% (414/414), done.


In [2]:
cd INNA-for-DeepLearning/INNA_for_keras/

/content/INNA-for-DeepLearning/INNA_for_keras


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
!pip install keras --upgrade 

  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [5]:
!pip install keras==2.2.4

     |████████████████████████████████| 317kB 10.8MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [6]:
import keras
import tensorflow as tf

print(keras.__version__)
tf.__version__

Using TensorFlow backend.


2.2.4


'1.15.2'

In [7]:
####################  Example on a toy problem ####################

# Essential packages

import numpy as np
np.random.seed(2) #27199925
import keras
from keras import backend as K
import pandas as pd
import csv



# Import the optimizer
from inna import *
from nadi2 import *

inna = INNA(lr=1e-1,alpha=0.1,beta=0.1,speed_ini=1.,decay=1.,decaypower=1./4)
#inna = NADIAN(lr=1e-1,alpha=0.1,beta=0.1,mu=0.9,speed_ini=1.,decay=1.,decaypower=1./4)
nadi = NADIAN(lr=1e-1,alpha=0.02,beta=0.9,mu=0.9,speed_ini=1.,decay=1.,decaypower=1./4)

# DATASET:

#from keras.datasets import cifar10
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_classes = 10
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

x_train = x_train.reshape(-1, 28,28, 1)
x_test = x_test.reshape(-1, 28,28, 1)

# Toy Network:

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

model = Sequential()
'''
model.add(Conv2D(filters = 6, 
                 kernel_size = 5, 
                 strides = 1, 
                 activation = 'relu', 
                 input_shape = (32,32,3)))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Conv2D(filters = 16, 
                 kernel_size = 5,
                 strides = 1,
                 activation = 'relu',
                 input_shape = (14,14,6)))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
#Flatten
model.add(Flatten())
model.add(Dense(units = 120, activation = 'relu'))
model.add(Dense(units = 84, activation = 'relu'))
#Output Layer
model.add(Dense(units = 10, activation = 'softmax'))
'''
#'''
#MNIST Network
model.add(Conv2D(filters =32, kernel_size = 5,
                 activation='relu',
                 input_shape = (28,28,1)))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Conv2D(filters =64, kernel_size = 3, activation='relu'))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.5))

#Output Layer
model.add(Dense(units = 10, activation = 'softmax'))
#'''



# Compile the model with the optimizer:

#model.compile(optimizer=inna, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=nadi, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the Network:
epochs = 100 ; batchsize = 32
HIST = model.fit(x_train, y_train,
              batch_size=batchsize,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              verbose = 1)
              
# Plot the loss function

#loss = HIST.history['loss']
#accuracy = HIST.history['acc']

df = pd.DataFrame(HIST.history)
df.to_csv('everepoch.csv')
print(pd.DataFrame(HIST.history))


import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(HIST.history['acc'])
plt.plot(HIST.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig("Accuracy.png", format="png", dpi=300)
plt.show()

# Plot training & validation loss values
plt.plot(HIST.history['loss'])
plt.plot(HIST.history['val_loss'])
plt.yscale('log')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig("loss.png", format="png", dpi=300)
plt.show()

#plt.plot(np.log10(loss), label="Train_loss")
#plt.plot(np.log10(accuracy), label="Train_Acce")
#plt.show()

11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples
y_train shape: (60000,)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Train on 60000 samples, validate on 10000 samples
Epoch 1/100





49984/60000 [=======================>......] - ETA: 5s - loss: 9.5565 - acc: 0.1494

KeyboardInterrupt: ignored

In [9]:
import keras
import tensorflow as tf

print(keras.__version__)    #2.2.4
print(tf.__version__)       #1.15.2

!nvcc --version
!python -V                  #3.7.10

2.2.4
2.4.1
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176
Python 3.7.10
